# Multilabel classification on galaxy zoo dataset

https://www.kaggle.com/c/galaxy-zoo-the-galaxy-challenge/overview

### Data includes images with unique IDs (jpg files) and a table containing the probabilities of each class in the columns




### Read table with classes

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os 
import tarfile
import urllib 
import pandas as pd
import seaborn as sns
from glob import glob
 
# Use pandas to retrieve data file
#url = '' # use to fetch data file each time

# Read in CSV file with probabilities
probs = pd.read_csv("training_solutions_rev1.csv",index_col=0)
probs.head()

,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,Class5.1,...,Class9.3,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6
GalaxyID,,,,,,,,,,,,,,,,,,,,,
100008,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,0.0,...,0.000000,0.279952,0.138445,0.000000,0.000000,0.092886,0.0,0.0,0.0,0.325512
100023,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,0.0,...,0.018764,0.000000,0.131378,0.459950,0.000000,0.591328,0.0,0.0,0.0,0.000000
100053,0.765717,0.177352,0.056931,0.000000,0.177352,0.000000,0.177352,0.000000,0.177352,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
100078,0.693377,0.238564,0.068059,0.000000,0.238564,0.109493,0.129071,0.189098,0.049466,0.0,...,0.000000,0.094549,0.000000,0.094549,0.189098,0.000000,0.0,0.0,0.0,0.000000
100090,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000


## Read images

In [10]:
# display image
from skimage import data,io,filters
url = ""
imfiles = glob("images_training_rev1/*.jpg")
#print(imfiles)
help(io.imread)#(imfiles)
#imlist=[io.imread(f) for f in imfiles]
#imix = [f.split('/')[-1][:-4] for f in imfiles]
print(imix[0:5])
fig=plt.figure(figsize=(6,6))
plt.imshow(imlist[0])

Help on function imread in module skimage.io._io:

imread(fname, as_gray=False, plugin=None, **plugin_args)
    Load an image from file.
    
    Parameters
    ----------
    fname : string
        Image file name, e.g. ``test.jpg`` or URL.
    as_gray : bool, optional
        If True, convert color images to gray-scale (64-bit floats).
        Images that are already in gray-scale format are not converted.
    plugin : str, optional
        Name of plugin to use.  By default, the different plugins are
        tried (starting with imageio) until a suitable
        candidate is found.  If not given and fname is a tiff file, the
        tifffile plugin will be used.
    
    Other Parameters
    ----------------
    plugin_args : keywords
        Passed to the given plugin.
    
    Returns
    -------
    img_array : ndarray
        The different color bands/channels are stored in the
        third dimension, such that a gray-image is MxN, an
        RGB-image MxNx3 and an RGBA-image M

NameError: name 'imlist' is not defined

<Figure size 432x432 with 0 Axes>

## Add image pre-processing here

# Split into training, test set 

##### Use ```sklearn.model_selection.train_test_split``` or do stratified sampling with ```StratifiedShuffleSplit```?

In [ ]:
from sklearn.model_selection import train_test_split
#from sklearn.base import clone 

X_train, X_test, y_train, y_test = train_test_split(cancer.copy().drop("diagnosis",axis=1), cancer.copy().diagnosis,\
                                                    test_size=0.3, shuffle=True, random_state=1)
#train_set, test_test = StratifiedShuffleSplit(df, test_size=0.25, random_state=42)


##### Use ```sklearn.model_selection.train_test_split``` or do stratified sampling with ```StratifiedShuffleSplit```?


#### Stochastic Gradient Descent classifier ```sklearn.linear_model.SGDClassifier```: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier

Set ```loss="hinge"``` fits a linear SVC (```sklearn.svm.LinearSVC```) but doesn't allow for probability estimates. Use ```loss="log"``` or ```"modified_huber"```

In [ ]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='modified_huber',max_iter=1e5,tol=1.0e-3,\
                   validation_fraction=0.3,early_stopping=True,shuffle=True,average=True)

#### SGD is sensitive to feature scaling... scale so features have mean=0 and variance=0 using ```StandardScaler()```

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### Fit SGD classifier to training set, and do cross-validation:
https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation

In [ ]:
clf.fit(X_train_scaled, y_train)

In [ ]:
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.metrics import roc_curve,precision_score,recall_score,f1_score,\
            average_precision_score,precision_recall_curve,confusion_matrix 

# First find scores from fitting (no cv)
scores = clf.decision_function( X_train_scaled)
# use CV for better evaluation
cv_scores = cross_val_score(clf, X_train_scaled, y_train, cv=3, scoring='accuracy')
y_scores = cross_val_predict(clf, X_train_scaled, y_train, cv=3, method='decision_function')
y_train_pred = cross_val_predict(clf, X_train_scaled, y_train, cv=3)
Cij = confusion_matrix( y_train, y_train_pred)
fpr,tpr,thresh = roc_curve( y_train, y_scores, pos_label='M')
precision,recall,thresholds = precision_recall_curve( y_train, y_scores, pos_label='M')

In [ ]:
#print("Avg score (train, test): ",score,clf.score)
print("Training cross-validation scores: ",cv_scores)
print("Training CV confusion matrix: ",Cij)
#print("Test scores: ",y_scores, scores)
# print("Test confusion matrix: ",Cij_test)

plt.figure(figsize=(10,10))
plt.plot(thresholds,precision[:-1],'m-')
plt.plot(thresholds,recall[:-1],'g-')
plt.xlabel("Thresh")
plt.ylabel("Rate")
#plt.xscale('log')
plt.xlim(-500,1000)
plt.ylim(0.2,1.1)
plt.legend(["Precision","Recall"])
#plt.annotate("Train set CV",xy=(0.6,0.6),xycoords="axes fraction")
plt.show()
plt.figure(figsize=(6,6))
plt.plot(fpr,tpr,'k-')
plt.xscale('log')
plt.xlabel("FPR")
plt.ylabel("TPR")
# plt.plot(thresh,tpr,'g-')
# plt.annotate("Train set CV",xy=(0.6,0.6),xycoords="axes fraction")
plt.show()
# plt.figure()
# plt.plot(thresh_test,fpr_test,'m-')
# plt.plot(thresh_test,tpr_test,'g-')
# plt.annotate("Test set",xy=(0.6,0.6),xycoords="axes fraction")
# plt.xlabel("FPR")
# plt.ylabel("TPR")
# plt.show()

### QUESTION: how do you predict probability of classification?

### Evaluate classifier using the test set

In [ ]:
y_test_pred = clf.predict(X_test_scaled)
y_test_scores = clf.decision_function(X_test_scaled)
y_test_score = clf.score( X_test_scaled, y_test)

Cij_test = confusion_matrix( y_test, y_test_pred)
cv_test_scores = cross_val_score(clf, X_test_scaled, y_test, cv=3, scoring='accuracy')

print(Cij_test)
print(cv_test_scores)
print(" ".join(Cij_test.ravel().astype('str')))

### TEST SET RESULTS: 

CONFUSION MATRIX:

In [ ]:
from IPython.display import display, Math
display(Math(r'%i %i \\ %i %i'.format(Cij_test)))#(Cij_test[0,0],Cij_test[0,1],Cij_test[1,0],Cij_test[1,1])))